# 加速：非同步爬蟲

* 了解非同步爬蟲加速原理與實作

## 作業目標

* 比較一下非同步爬蟲跟多線程爬蟲的差異是什麼？各自的優缺點為何？

In [9]:
'''
Your Input
'''
#非同步爬蟲

import aiohttp, asyncio
import nest_asyncio
import requests, time

URL = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'
nest_asyncio.apply()

async def job(session):
    response = await session.get(URL)                               #等待並切換
    return str(response.url)

async def main(loop):
    async with aiohttp.ClientSession() as session:                  #官網推薦建立Session的形式,也可以直接用request
        tasks = [loop.create_task(job(session)) for _ in range(5)]
        finished, unfinished = await asyncio.wait(tasks)            #收集完成的結果,會返回完成的和沒完成的,等全部都完成了才返回
        all_results = [r.result() for r in finished]                #獲取所有結果
        print(all_results)

t1 = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
# loop.close()
print("Async total time:", time.time() - t1)

['https://rate.bot.com.tw/xrt?Lang=zh-TW', 'https://rate.bot.com.tw/xrt?Lang=zh-TW', 'https://rate.bot.com.tw/xrt?Lang=zh-TW', 'https://rate.bot.com.tw/xrt?Lang=zh-TW', 'https://rate.bot.com.tw/xrt?Lang=zh-TW']
Async total time: 2.8506598472595215


In [2]:
#多線程爬蟲

import requests
import _thread
import time

urls = ['https://rate.bot.com.tw/xrt?Lang=zh-TW', 'https://rate.bot.com.tw/xrt?Lang=zh-TW', 'https://rate.bot.com.tw/xrt?Lang=zh-TW',
        'https://rate.bot.com.tw/xrt?Lang=zh-TW', 'https://rate.bot.com.tw/xrt?Lang=zh-TW']

startTime = time.time()

for url in urls:
    _thread.start_new_thread(requests.get, (url,))
    print(url)

finishTime = time.time()
print(finishTime - startTime) # 利用 thread 的爬蟲所需時間

https://rate.bot.com.tw/xrt?Lang=zh-TW
https://rate.bot.com.tw/xrt?Lang=zh-TW
https://rate.bot.com.tw/xrt?Lang=zh-TW
https://rate.bot.com.tw/xrt?Lang=zh-TW
https://rate.bot.com.tw/xrt?Lang=zh-TW
0.0008039474487304688


Unclosed connection
client_connection: Connection<ConnectionKey(host='morvanzhou.github.io', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>


> 線程較省資源。

> 多線程和多進程的缺點是在IO阻塞時會造成了線程和進程的浪費，所以非同步IO是更好的方式。 非同步IO請求的本質則是非阻塞Socket + IO多路復用。這裡只需要一個線程，而每一個請求則是一個協程。 

>資料來源：https://codingnote.cc/zh-tw/p/49958